# Ch `02`: Concept `09`

## Using Queues

If you have a lot of training data, you probably don't want to load it all into memory at once. The QueueRunner in TensorFlow is a tool to efficiently employ a queue data-structure in a multi-threaded way.

In [2]:
import tensorflow as tf
import numpy as np

We will be running multiple threads, so let's figure out the number of CPUs:

In [3]:
import multiprocessing
NUM_THREADS = multiprocessing.cpu_count()

Generate some fake data to work with:

In [7]:
xs = np.random.randn(100, 3)
print(xs.shape)

ys = np.random.randint(0, 2, size=100)

(100, 3)


Here's a couple concrete examples of our data:

In [8]:
xs_and_ys = zip(xs, ys)
for _ in range(5):
    x, y = next(xs_and_ys)
    print('Input {}  --->  Output {}'.format(x, y))

Input [ 0.08234752 -0.08884229  0.10660199]  --->  Output 0
Input [ 1.6462786  -0.46028431 -0.67948008]  --->  Output 0
Input [0.98023512 0.39208894 0.5936347 ]  --->  Output 0
Input [-0.25953881  0.30116748 -0.15760827]  --->  Output 1
Input [ 0.02039786 -1.42820193  1.54763609]  --->  Output 0


Define a queue:

In [9]:
queue = tf.FIFOQueue(capacity=1000, dtypes=[tf.float32, tf.int32])

Set up the enqueue and dequeue ops:

In [10]:
enqueue_op = queue.enqueue_many([xs, ys])
x_op, y_op = queue.dequeue()

Define a QueueRunner:

In [11]:
qr = tf.train.QueueRunner(queue, [enqueue_op] * 4)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Now that all variables and ops have been defined, let's get started with a session:

In [12]:
sess = tf.InteractiveSession()

Create threads for the QueueRunner:

In [13]:
coord = tf.train.Coordinator()
enqueue_threads = qr.create_threads(sess, coord=coord, start=True)

Test out dequeueing:

In [14]:
for _ in range(100):
    if coord.should_stop():
        break
    x, y = sess.run([x_op, y_op])
    print(x, y)
coord.request_stop()
coord.join(enqueue_threads)

[ 0.08234752 -0.0888423   0.10660198] 0
[ 1.6462786  -0.46028432 -0.6794801 ] 0
[0.9802351  0.39208895 0.5936347 ] 0
[-0.2595388   0.3011675  -0.15760827] 1
[ 0.02039786 -1.4282019   1.547636  ] 0
[-1.4833885   1.0614552   0.22581042] 1
[-0.34202048 -0.27590343  0.56887305] 0
[ 0.66067773 -0.39466104  1.2408216 ] 1
[ 2.172815  -1.0601367  0.1477568] 1
[-0.29749846 -0.670709   -0.50491506] 0
[-0.20297597 -1.2790078   0.4204365 ] 1
[-1.9370341   1.0771289   0.21904531] 1
[-0.05158934 -0.3923433  -1.3717874 ] 0
[-0.1322114 -0.1445082  0.4646972] 1
[ 1.0406339 -1.1494099 -0.9475374] 0
[0.43618202 0.2844854  1.8792194 ] 1
[-0.18623014 -0.68094003  0.7782209 ] 1
[-0.98051006  1.0867769   0.53598917] 1
[ 1.8654356  -0.84322643  0.50363576] 1
[-1.4713945   0.21914394 -0.9167111 ] 1
[ 2.327208   -0.16346677  1.175957  ] 0
[-1.0443866   0.59765595  1.6381918 ] 0
[ 2.027856   0.769454  -1.7665235] 0
[-0.36451837  0.9780361  -0.4567181 ] 0
[ 0.6334522  -0.17396022  0.42654067] 1
[0.13931692 1.1457